# Đồ án cuối kỳ

Họ tên: Võ Nhật Thịnh

MSSV: 1712796


# 1. nguồn dữ liệu

>Khái quát về dữ liệu:

Bộ dữ liệu cung cấp thông tin tình hình dịch COVID-19 đang diễn ra trên thế giới, mức độ lây lan của chúng, tổng số ca mắc, ca qua khỏi và ca tử vong theo thời gian thực, tức bộ dữ liệu luôn được cập nhật từng ngày từng giờ giúp ta có 1 cái nhìn trực quan nhất về dịch

Nguồn dữ liệu: https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset

>license của chủ đề:

Bộ dữ liệu có bản quyền thuộc về Creative Commons Attribution 4.0 International (CC BY 4.0) của trường Johns Hopkins University
để sử dụng bộ dữ liệu cho mục đích nghiên cứu, ta trích dẫn nguồn dữ liệu:

"Dong E, Du H, Gardner L. An interactive web-based dashboard to track COVID-19 in real time. Lancet Inf Dis. 20(5):533-534. doi: 10.1016/S1473-3099(20)30120-1"

chi tiết tham khảo: https://github.com/CSSEGISandData/COVID-19/blob/master/README.md

> cách dữ liệu được thu thập:

nguồn dữ liệu được thu thập từ 3 nguồn chính: 

    data tổng hợp từ các tổ chức lớn trên thế giới (WHO, ECDC, DXY,..)
    
    data từ các bang của US
    
    data từ các nước/ vùng khác ngoài US

# 2. khám phá dữ liệu

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

ở đây bộ dữ liệu gồm 8 file, ta tiến hành xem qua từng để có 1 cái nhìn tổng quan cấu trúc file dữ liệu. ta dễ dàng thấy bộ dữ liệu gồm 2 loại file: 

tập các file time series: thể hiện các loại ca mắc bệnh theo ngày

tập các file COVID-19: chi tiết nhật ký ghi lại các ca mắc COVID-19

### Đọc dữ liệu vào dataframe

In [3]:
#nạp các file vào biến
covid_19_data = pd.read_csv('covid_19_data.csv')
COVID19_line_list_data = pd.read_csv('COVID19_line_list_data.csv')
COVID19_open_line_list = pd.read_csv('COVID19_open_line_list.csv')

time_series_covid_19_confirmed = pd.read_csv('time_series_covid_19_confirmed.csv')
time_series_covid_19_confirmed_US = pd.read_csv('time_series_covid_19_confirmed_US.csv')
time_series_covid_19_deaths = pd.read_csv('time_series_covid_19_deaths.csv')
time_series_covid_19_deaths_US = pd.read_csv('time_series_covid_19_deaths_US.csv')
time_series_covid_19_recovered = pd.read_csv('time_series_covid_19_recovered.csv')

Vì đây là bộ dữ liệu được tổng hợp từ nhiều nguồn nên khả năng dữ liệu bị lỗi hay missing value thì không thể tránh được. với khả năng hiện tại thì ta chỉ xử lý lỗi missing value

In [4]:
# file covid_19_data.csv:
f = covid_19_data
miss_f = [(c, f[c].isna().mean()*100) for c in f]
miss_f = pd.DataFrame(miss_f, columns=["column_name", "percentage"])
miss_f

,column_name,percentage
0,SNo,0.00000
1,ObservationDate,0.00000
2,Province/State,34.14228
3,Country/Region,0.00000
4,Last Update,0.00000
5,Confirmed,0.00000
6,Deaths,0.00000
7,Recovered,0.00000


In [5]:
f = COVID19_line_list_data
miss_f = [(c, f[c].isna().mean()*100) for c in f]
miss_f = pd.DataFrame(miss_f, columns=["column_name", "percentage"])
miss_f

,column_name,percentage
0,id,0.000000
1,case_in_country,18.156682
2,reporting date,0.092166
3,Unnamed: 3,100.000000
4,summary,0.460829
5,location,0.000000
6,country,0.000000
7,gender,16.866359
8,age,22.304147
9,symptom_onset,48.110599


In [6]:
f = COVID19_open_line_list
miss_f = [(c, f[c].isna().mean()*100) for c in f]
miss_f = pd.DataFrame(miss_f, columns=["column_name", "percentage"])
miss_f

,column_name,percentage
0,ID,6.746425
1,age,90.450234
2,sex,91.051961
3,city,27.835198
4,province,8.636557
5,country,6.923404
6,wuhan(0)_not_wuhan(1),6.767662
7,latitude,6.930483
8,longitude,6.930483
9,geo_resolution,6.930483


In [7]:
f = time_series_covid_19_confirmed
miss_f = [(c, f[c].isna().mean()*100) for c in f]
miss_f = pd.DataFrame(miss_f, columns=["column_name", "percentage"])
miss_f

,column_name,percentage
0,Province/State,69.548872
1,Country/Region,0.000000
2,Lat,0.000000
3,Long,0.000000
4,1/22/20,0.000000
...,...,...
192,7/28/20,0.000000
193,7/29/20,0.000000
194,7/30/20,0.000000
195,7/31/20,0.000000


In [8]:
f = time_series_covid_19_confirmed_US
miss_f = [(c, f[c].isna().mean()*100) for c in f]
miss_f = pd.DataFrame(miss_f, columns=["column_name", "percentage"])
miss_f

,column_name,percentage
0,UID,0.000000
1,iso2,0.000000
2,iso3,0.000000
3,code3,0.000000
4,FIPS,0.299401
...,...,...
199,7/28/20,0.000000
200,7/29/20,0.000000
201,7/30/20,0.000000
202,7/31/20,0.000000


In [9]:
f = time_series_covid_19_deaths
miss_f = [(c, f[c].isna().mean()*100) for c in f]
miss_f = pd.DataFrame(miss_f, columns=["column_name", "percentage"])
miss_f

,column_name,percentage
0,Province/State,69.548872
1,Country/Region,0.000000
2,Lat,0.000000
3,Long,0.000000
4,1/22/20,0.000000
...,...,...
192,7/28/20,0.000000
193,7/29/20,0.000000
194,7/30/20,0.000000
195,7/31/20,0.000000


In [10]:
f = time_series_covid_19_deaths_US
miss_f = [(c, f[c].isna().mean()*100) for c in f]
miss_f = pd.DataFrame(miss_f, columns=["column_name", "percentage"])
miss_f

,column_name,percentage
0,UID,0.000000
1,iso2,0.000000
2,iso3,0.000000
3,code3,0.000000
4,FIPS,0.299401
...,...,...
200,7/28/20,0.000000
201,7/29/20,0.000000
202,7/30/20,0.000000
203,7/31/20,0.000000


In [11]:
f = time_series_covid_19_recovered
miss_f = [(c, f[c].isna().mean()*100) for c in f]
miss_f = pd.DataFrame(miss_f, columns=["column_name", "percentage"])
miss_f

,column_name,percentage
0,Province/State,73.517787
1,Country/Region,0.000000
2,Lat,0.000000
3,Long,0.000000
4,1/22/20,0.000000
...,...,...
192,7/28/20,0.000000
193,7/29/20,0.000000
194,7/30/20,0.000000
195,7/31/20,0.000000


như ta đã dự đoán từ trước, bộ dữ liệu trên có rất nhiều giá trị missing value, để tránh việc khó khăn trong phân tích dữ liệu, ta sẽ lấy file có độ lỗi nhỏ nhất để phân tích, ở đây ta sử dụng file covid_19_data.csv

tiếp theo ta điền các giá trị missing value cho file covid_19_data.csv:

In [14]:
covid_19_data["Province/State"]= covid_19_data["Province/State"].fillna('Unknown')

### thể hiện tổng quan về file

In [19]:
covid_19_data = pd.read_csv('covid_19_data.csv')
covid_19_data.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


### các thuộc tính của data

In [22]:
rows, columns =covid_19_data.shape
print("num rows: {}, num column: {} (data in 27/08/2020)".format(rows,columns))
covid_19_data.info()

num rows: 77467, num column: 8 (data in 27/08/2020)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77467 entries, 0 to 77466
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SNo              77467 non-null  int64  
 1   ObservationDate  77467 non-null  object 
 2   Province/State   51018 non-null  object 
 3   Country/Region   77467 non-null  object 
 4   Last Update      77467 non-null  object 
 5   Confirmed        77467 non-null  float64
 6   Deaths           77467 non-null  float64
 7   Recovered        77467 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 4.7+ MB


chi tiết các thuộc tính:

&emsp;&emsp;+ **SNo**: Số se-ri <br>
&emsp;&emsp;+ **ObservationDate**: ngày phát hiện theo định dạng MM/DD/YYYY <br>
&emsp;&emsp;+ **Province/State**: tỉnh/bang phát hiện ca mắc <br>
&emsp;&emsp;+ **Country/Region**: quốc gia phát hiện ca mắc <br>
&emsp;&emsp;+ **Last Update**: thời gian ghi nhận ca mắc theo múi giờ UTC sở tại <br>
&emsp;&emsp;+ **Confirmed**: lũy kế số ca mắc được xác nhận <br>
&emsp;&emsp;+ **Deaths**: lũy kế số ca mắc không qua khỏi <br>
&emsp;&emsp;+ **Recovered**: lũy kế số ca mắc đã hồi phục <br>

mỗi dòng trong file có ý nghĩa: 

# 3. Đưa ra các câu hỏi cần trả lời

1. Tình hình dịch tại Trung Quốc đang diễn biến như thế nào?
2. số ca nhiễm đang chữa trị tại thời điểm hiện tại
3. tiếp tục tình hình dịch như thế này dự đoán số người chết tại Việt Nam tăng lên bao nhiêu trong 1 tháng tới

---------------